In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.2.4
2.5


In [3]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [4]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132)
RealParameter('A.1_Bmax', 30, 350)
RealParameter('A.1_pfail', 0, 1)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350)
RealParameter('A.2_pfail', 0, 1)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350)
RealParameter('A.3_pfail', 0, 1)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350)
RealParameter('A.4_pfail', 0, 1)
CategoricalParameter('A.4_Brate', [0, 1, 2])
RealParameter('A.5_Bmax', 30, 350)
RealParameter('A.5_pfail', 0, 1)
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [6]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1)
IntegerParameter('0_RfR 1', 0, 1)
IntegerParameter('0_RfR 2', 0, 1)
IntegerParameter('1_RfR 0', 0, 1)
IntegerParameter('1_RfR 1', 0, 1)
IntegerParameter('1_RfR 2', 0, 1)
IntegerParameter('2_RfR 0', 0, 1)
IntegerParameter('2_RfR 1', 0, 1)
IntegerParameter('2_RfR 2', 0, 1)
IntegerParameter('3_RfR 0', 0, 1)
IntegerParameter('3_RfR 1', 0, 1)
IntegerParameter('3_RfR 2', 0, 1)
IntegerParameter('4_RfR 0', 0, 1)
IntegerParameter('4_RfR 1', 0, 1)
IntegerParameter('4_RfR 2', 0, 1)
IntegerParameter('EWS_DaysToThreat', 0, 4)
IntegerParameter('A.1_DikeIncrease 0', 0, 10)
IntegerParameter('A.1_DikeIncrease 1', 0, 10)
IntegerParameter('A.1_DikeIncrease 2', 0, 10)
IntegerParameter('A.2_DikeIncrease 0', 0, 10)
IntegerParameter('A.2_DikeIncrease 1', 0, 10)
IntegerParameter('A.2_DikeIncrease 2', 0, 10)
IntegerParameter('A.3_DikeIncrease 0', 0, 10)
IntegerParameter('A.3_DikeIncrease 1', 0, 10)
IntegerParameter('A.3_DikeIncrease 2', 0, 10)
IntegerParameter('A.4_DikeIncreas

In [7]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=['A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'], function=<function sum_over at 0x0000025F5F4C4678>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=['A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'], function=<function sum_over at 0x0000025F5F4C4678>)
ScalarOutcome('A.2 Total Costs', variable_name=['A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'], function=<function sum_over at 0x0000025F5F4C4678>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=['A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'], function=<function sum_over at 0

In [8]:
from ema_workbench import Policy

In [9]:
# policy_dict = {}
# for i in range(5):
#     for j in range(3):
#         policy_dict[f'{i}_RfR {j}'] = 0
# policy_dict['EWS_DaysToThreat'] = 0
# for i in range(1,6):
#     for j in range(3):
#         policy_dict[f'A.{i}_DikeIncrease {j}'] = 5
# policy_dict 

# policy = [Policy('only_rfr', **policy_dict)]

# #pass the policies list to EMA workbench experiment runs
# n_scenarios = 5000
# with MultiprocessingEvaluator(dike_model) as evaluator:
#     experiments, outcomes = evaluator.perform_experiments(n_scenarios,
#                                             policy)

# from ema_workbench.util import utilities
# utilities.save_results((experiments, outcomes), 'dike_5_results.tar.gz')

## Optimization

In [10]:
# load the basecase results
from ema_workbench import load_results
experiments, outcomes = load_results('../results/basecase_results.tar.gz') 
outcomes_df = pd.DataFrame.from_dict(outcomes)

[MainProcess/INFO] results loaded succesfully from D:\Willy\Documents\GitHub\Model_based_decision_making\EPA1361\final assignment\results\basecase_results.tar.gz


In [11]:
# function to aggregate over time and locations
def aggregate_df(df):
    df_aggregate_time = pd.DataFrame()
    df_aggregate_time_location = pd.DataFrame()
    locations = ['A.1', 'A.2', 'A.3', 'A.4', 'A.5']
    step = ['0', '1', '2']
    metrics = ['Expected Annual Damage', 'Dike Investment Costs', 'Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs']
    
    for metric in metrics:
        if metric == "RfR Total Costs" or metric == "Expected Evacuation Costs":

            columns = [metric + ' ' + time for time in step]

            df_aggregate_time_location[metric + ' time aggregate'] = outcomes_df[columns].sum(axis=1)
        else:
            for location in locations:
                columns = [location + '_' + metric + ' ' + time for time in step]

                df_aggregate_time[location + '_' + metric + ' time aggregate'] = outcomes_df[columns].sum(axis=1)
            
            columns_locations = [location + '_' + metric + ' time aggregate' for location in locations]
            df_aggregate_time_location[metric + ' time location aggregate'] = df_aggregate_time[columns_locations].sum(axis=1)
                    

    return df_aggregate_time, df_aggregate_time_location

{'A.1_Expected Annual Damage 0': array([1.01340901e+09, 6.60904821e+08, 8.08000176e+08, ...,
        8.10923526e+08, 6.63058641e+08, 1.32416076e+09]),
 'A.1_Dike Investment Costs 0': array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 'A.1_Expected Number of Deaths 0': array([0.64368097, 0.64245025, 0.64435081, ..., 0.64728549, 0.64509982,
        0.64971976]),
 'A.2_Expected Annual Damage 0': array([       0.        ,  2246637.42799311,   423014.02316611, ...,
               0.        , 14985252.53698277,        0.        ]),
 'A.2_Dike Investment Costs 0': array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 'A.2_Expected Number of Deaths 0': array([0.        , 0.00306913, 0.00049785, ..., 0.        , 0.02026549,
        0.        ]),
 'A.3_Expected Annual Damage 0': array([0.00000000e+00, 4.51796930e+08, 3.77539485e+08, ...,
        2.04215963e+07, 4.57979724e+08, 0.00000000e+00]),
 'A.3_Dike Investment Costs 0': array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 'A.3_Expected Number of Deaths 0': ar

In [27]:
# Instantiate the model
dike_model, planning_steps = get_model_for_problem_formulation(3)

# Redefine the uncertainties
dike_model.uncertainties['A.1_pfail'] = RealParameter('A.1_pfail', 0.000120, 0.414252)
dike_model.uncertainties['A.3_pfail'] = RealParameter('A.3_pfail', 0.000010, 0.181847)
dike_model.uncertainties['A.1_Bmax'] = RealParameter('A.1_Bmax', 30.012525, 298.534935)
#dike_model.uncertainties['A.4_Bmax'] = RealParameter('A.4_Bmax', 39.275594, 349.965209)

In [32]:
def s_to_n(*data):
    mean = np.mean(data)
    std = np.std(data)
    return mean*std

In [33]:
from ema_workbench import (MultiprocessingEvaluator, Policy, Scenario, Constraint)


#Making lists of all the disaggregated variables in order to properly use the total aggregated variables

#rfr_costs = ['RfR Total Costs 0', 'RfR Total Costs 1', 'RfR Total Costs 2']


evac_costs = ['Expected Evacuation Costs']


total_costs = ['A.1 Total Costs', 'A.2 Total Costs','A.3 Total Costs','A.4 Total Costs', 'A.5 Total Costs']

#Specifying robustness functions. All have to be minimized. All except for the variance will use the 'signal_to_noise' function.
#Variance will use the 'signal_to_noise_variation' function, which contains the extra step of first calculating the standard deviation.
robustness_functions = [ScalarOutcome('Annual Costs Score', kind=ScalarOutcome.MINIMIZE, variable_name=total_costs, function=s_to_n),
                        ScalarOutcome('Evacuation Costs Score', kind=ScalarOutcome.MINIMIZE, variable_name=evac_costs, function=s_to_n)]#,
                        #ScalarOutcome('RfR Costs Score', kind=ScalarOutcome.MINIMIZE, variable_name=rfr_costs, function=s_to_n)]

max_num_deaths = 0.00001

# deaths = ['A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2',
#                    'A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2',
#                    'A.3_Expected Number of Deaths 0', 'A.3_Expected Number of Deaths 1', 'A.3_Expected Number of Deaths 2',
#                    'A.4_Expected Number of Deaths 0', 'A.4_Expected Number of Deaths 1', 'A.4_Expected Number of Deaths 2',
#                    'A.5_Expected Number of Deaths 0', 'A.5_Expected Number of Deaths 1', 'A.5_Expected Number of Deaths 2']

deaths = ['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths','A.3_Expected Number of Deaths','A.4_Expected Number of Deaths','A.5_Expected Number of Deaths']

#Adding the constraint for Room for the River costs.
constraints = [Constraint("max number of deaths A.1", outcome_names=deaths[0],
                          function=lambda x:max(0, x-max_num_deaths)),
               Constraint("max number of deaths A.2", outcome_names=deaths[1],
                          function=lambda x:max(0, x-max_num_deaths)),
               Constraint("max number of deaths A.3", outcome_names=deaths[2],
                          function=lambda x:max(0, x-max_num_deaths)),
               Constraint("max number of deaths A.4", outcome_names=deaths[3],
                          function=lambda x:max(0, x-max_num_deaths)),
               Constraint("max number of deaths A.5", outcome_names=deaths[4],
                          function=lambda x:max(0, x-max_num_deaths))]

In [36]:
from ema_workbench.em_framework import sample_uncertainties
n_scenarios = 50
scenarios = sample_uncertainties(dike_model, n_scenarios)

In [ ]:
from ema_workbench.em_framework.optimization import (HyperVolume, 
                                                     EpsilonProgress)
from ema_workbench.em_framework.evaluators import BaseEvaluator
import time

ema_logging.log_to_stderr(ema_logging.INFO)

convergence = [HyperVolume(minimum=[1e10, 500000, 1e10, 1e2], 
                           maximum=[1e20, 1000000, 1e20, 1e10]),
               EpsilonProgress()]
nfe = int(15000)

start = time.time()

with MultiprocessingEvaluator(dike_model) as evaluator:
    archive, convergence = evaluator.robust_optimize(robustness_functions, scenarios=n_scenarios, 
                                               nfe=nfe, convergence=convergence,
                                               epsilons=[0.01,1000000000,100000,1000000000,10000],
                                               constraint=constraints)
    

end = time.time()

print('Processing time:',(end-start)/60,'Minutes')

from ema_workbench.util import utilities
utilities.save_results((archive, convergence), 'optimisation_results.tar.gz')

[MainProcess/INFO] generation 0: 0/15000 nfe
[MainProcess/INFO] generation 5: 498/15000 nfe
[MainProcess/INFO] generation 10: 996/15000 nfe


### 